# Chinook: Analytics Business Decisions
In this project, we will be working with a database called Chinook. This database is associated with a digital music store. It contains the information about Employee, Customer, Invoice, Track, Playlist, Artist, Album,... The schema of data:
<br>
<img src="chinook-schema.png" style="width: 500px;">

## Connect jupyter notebook to our database file

In [1]:
# Import necessary library
import sqlite3
import pandas as pd
import plotly.express as px

In [5]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

- Declare all the name and type of table inside the database

In [6]:
%%sql
SELECT
    name, type
FROM sqlite_master
WHERE type IN ("table", "view")

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


There are totally 11 tables in database, I'll show 4 tables below as an example

In [7]:
%%sql
SELECT * FROM album
LIMIT 5

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [8]:
%%sql
SELECT * FROM customer
LIMIT 5

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [9]:
%%sql
SELECT * FROM invoice
LIMIT 5

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [10]:
%%sql
SELECT * FROM track
LIMIT 5

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


## Most purchases genre in USA

- The Chinook record store has just signed a deal with a new record label that specialized in artist of USA. These are their current gernes (artist)
<br>
- Artist: Regal (Hip-hop), Red Tone (Punk), Meteor and the Girls (Pop), Slim Jim Bites (Blues)
<br>
- In order to do that, I need to write a query that determines **three genres**, with the **most number of tracks sold in the USA**: in *absolute numbers* & *in percentage*

In [11]:
%%sql
/*The key is track ids sold in USA first!*/
WITH 
    USA_tracks AS 
    (
        SELECT track_id, quantity
        FROM invoice i
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
        WHERE i.billing_country = "USA"
    )

/* Return genre, number of purchases in absolute and percentage 
with most number of tracks sold in USA*/
SELECT 
    g.name genre,
    SUM(usat.quantity) tracks_sold,
    CAST(SUM(usat.quantity) AS FLOAT) / 
    (SELECT sum(il.quantity)
     FROM invoice i
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
     WHERE i.billing_country = "USA") AS percentage_sold
FROM track t
    INNER JOIN USA_tracks usat ON t.track_id = usat.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 3 DESC

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In [12]:
query = """WITH 
    USA_tracks AS 
    (
        SELECT track_id, quantity
        FROM invoice i
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
        WHERE i.billing_country = "USA"
    )

/* Return genre, number of purchases in absolute and percentage 
with most number of tracks sold in USA*/
SELECT 
    g.name genre,
    SUM(usat.quantity) tracks_sold,
    CAST(SUM(usat.quantity) AS FLOAT) / 
    (SELECT sum(il.quantity)
     FROM invoice i
        INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
     WHERE i.billing_country = "USA") AS percentage_sold
FROM track t
    INNER JOIN USA_tracks usat ON t.track_id = usat.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 3 DESC"""

# Make connection to database
data = sqlite3.connect("chinook.db")

# Convert query output into dataframe
gerne_data = pd.read_sql_query(query, data)
gerne_data = gerne_data.append(gerne_data.loc[9:, ["tracks_sold", "percentage_sold"]].sum(),
                               ignore_index=True)
gerne_data.iloc[-1, 0] = "Other"
# Drop uneeded rows
gerne_data.drop(index=range(9,17), inplace=True)

# Create a pie plot for the result
fig = px.pie(data_frame=gerne_data, names="genre", values="percentage_sold",
            title="Percentage tracks sold of each gerne in USA",
            color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_layout(legend_title_text="Genre")
fig.show()


After performing some query, I have some remark for the USA music's market field
<br>
- **"Rock"** is by far the most popular music genre in USA which accounts for around 50% of the market
<br>
- By following "Rock", both "Alternative & Punk" and "metal" nearly equal 1/10 the total number of music purchases each in USA
<br>
**Recommendation**: "Rock", "Alternative & Punk" and "metal"

## Sale support agent productivity

- Write a query that finds the **total dollar amount of sales assigned to each sales support agent** within the company

In [13]:
%%sql
WITH
    /*Find info of Sale support agent (an employee) and their customer info*/
    sale_agent AS
    (
        SELECT
            e.employee_id,
            e.first_name || " " || e.last_name AS employee_name,
            e.title AS position,
            e.country,
            c.customer_id,
            c.first_name || " " || c.last_name AS customer_name,
            c.country,
            e.hire_date
        FROM employee e
            INNER JOIN customer c ON e.employee_id = c.support_rep_id
        WHERE e.title LIKE "Sales Support Agent"
    )
    
/*Find total dollards amount of sales assigned to each Agent*/
SELECT 
    sa.employee_id AS 'Employee ID',
    sa.employee_name AS "Employee Name",
    sa.country AS 'Country',
    sa.position AS 'Position',
    sa.hire_date AS 'Date hired',
    ROUND(SUM(i.total),2) AS 'Total Sales'
FROM invoice i 
    INNER JOIN sale_agent sa ON i.customer_id = sa.customer_id
GROUP BY 1,2

 * sqlite:///chinook.db
Done.


Employee ID,Employee Name,Country,Position,Date hired,Total Sales
3,Jane Peacock,Canada,Sales Support Agent,2017-04-01 00:00:00,1731.51
4,Margaret Park,Canada,Sales Support Agent,2017-05-03 00:00:00,1584.0
5,Steve Johnson,Canada,Sales Support Agent,2017-10-17 00:00:00,1393.92


In [14]:
query = """WITH
    /*Find info of Sale support agent (an employee) and their customer info*/
    sale_agent AS
    (
        SELECT
            e.employee_id,
            e.first_name || " " || e.last_name AS employee_name,
            e.title AS position,
            e.country,
            c.customer_id,
            c.first_name || " " || c.last_name AS customer_name,
            c.country
        FROM employee e
            INNER JOIN customer c ON e.employee_id = c.support_rep_id
        WHERE e.title LIKE "Sales Support Agent"
    )
    
/*Find total dollards amount of sales assigned to each Agent*/
SELECT 
    sa.employee_id AS 'Employee ID',
    sa.employee_name AS "Employee Name",
    sa.country AS 'Country',
    sa.position AS 'Position',
    ROUND(SUM(i.total),2) AS 'Total Sales'
FROM invoice i 
    INNER JOIN sale_agent sa ON i.customer_id = sa.customer_id
GROUP BY 1,2"""

# Connect to database
data = sqlite3.connect("chinook.db")

# Transfer result query to dataframe
employee_sale = pd.read_sql_query(query, data)

# Sketch bar plot the total sales
fig = px.bar(data_frame=employee_sale, x="Employee Name",
            y="Total Sales", title="Total sales of sale agent employee",
            text="Total Sales",
            hover_data=["Employee ID", "Employee Name", "Country", "Position", "Total Sales"])
fig.update_yaxes(showticklabels=False)
fig.update_layout(plot_bgcolor="rgba(0,0,0,0)")
fig.update_traces(marker_color="rgb(207,207,207)") # Change color of bar to rgb(207,207,207)
fig.show()

- There are totally 3 Sale Support Agent and all of them made around \$4500 dollard for the company 
<br>
- The plot displays that "Jane Peacock" is the top Sale Support Agent who produces \$1732. Following are "Margaret Park and Steve Johnson respectively 

## Analyse sale data on different countries

- It's very important to find out which country make the most profits for chinook. 
<br>

The statistics of analysing on Chinook depending on these fields:
<br>
- "Number of customer" -> Number of customers in each country
- "Total sales Amount" -> Total amount of money spent by customers
- "Average order Amount" -> The average amount per purchase by customers
- "Average customer sale" -> The average value of sales per customer 

<br>
Note that there are some countries has more than 2 customers when some have only 1 customer that can be considered as "Other" group

In [15]:
%%sql
WITH 
    /* Generate with country col contained "Other" value*/
    country_or_other AS
    (
        /*Change all country with number of customer = 1 to "Other"*/
        SELECT CASE 
            WHEN 
                (
                    SELECT COUNT(*) 
                    FROM customer 
                    WHERE country = c.country
                    GROUP BY country
                ) = 1 THEN "Other"
            ELSE c.country
            END AS country,
            i.*
        FROM customer c
            INNER JOIN invoice i ON i.customer_id = c.customer_id
    )
    
    
SELECT 
    gg_sort.country 'Country',
    COUNT(DISTINCT customer_id) 'No Of Customers', 
    ROUND(CAST(SUM(total) AS FLOAT), 2) 'Total Sales',
    ROUND(CAST(SUM(total) AS FLOAT) / COUNT(DISTINCT customer_id), 2) 'Average Customer Sales',
    ROUND(CAST(AVG(total) AS FLOAT), 2) 'Average Order Amount',
    gg_sort.sort
FROM 
    (
        SELECT gg.*,
            CASE WHEN 
                country = "Other" THEN 1
                ELSE 0
            END AS sort 
        FROM country_or_other gg
    ) gg_sort
GROUP BY country
ORDER BY sort, 3 DESC


 * sqlite:///chinook.db
Done.


Country,No Of Customers,Total Sales,Average Customer Sales,Average Order Amount,sort
USA,13,1040.49,80.04,7.94,0
Canada,8,535.59,66.95,7.05,0
Brazil,5,427.68,85.54,7.01,0
France,5,389.07,77.81,7.78,0
Germany,4,334.62,83.66,8.16,0
Czech Republic,2,273.24,136.62,9.11,0
United Kingdom,3,245.52,81.84,8.77,0
Portugal,2,185.13,92.57,6.38,0
India,2,183.15,91.58,8.72,0
Other,15,1094.94,73.0,7.45,1


In [16]:
query = """WITH 
    /* Generate with country col contained "Other" value*/
    country_or_other AS
    (
        /*Change all country with number of customer = 1 to "Other"*/
        SELECT CASE 
            WHEN 
                (
                    SELECT COUNT(*) 
                    FROM customer 
                    WHERE country = c.country
                    GROUP BY country
                ) = 1 THEN "Other"
            ELSE c.country
            END AS country,
            i.*
        FROM customer c
            INNER JOIN invoice i ON i.customer_id = c.customer_id
    )
    
    
SELECT 
    gg_sort.country 'Country',
    COUNT(DISTINCT customer_id) 'No Of Customers', 
    ROUND(CAST(SUM(total) AS FLOAT), 2) 'Total Sales',
    ROUND(CAST(SUM(total) AS FLOAT) / COUNT(DISTINCT customer_id), 2) 'Average Customer Sales',
    ROUND(CAST(AVG(total) AS FLOAT), 2) 'Average Order Amount',
    gg_sort.sort
FROM 
    (
        SELECT gg.*,
            CASE WHEN 
                country = "Other" THEN 1
                ELSE 0
            END AS sort 
        FROM country_or_other gg
    ) gg_sort
GROUP BY country
ORDER BY sort, 3"""

# Connect to database
data = sqlite3.connect("chinook.db")

# Tranfer sql result to dataframe
data = pd.read_sql_query(query, data)
data.drop(columns="sort", inplace=True)

# Use bar chart to visualize the total sale of each country
fig = px.bar(data_frame = data, x="Total Sales",
            y="Country", orientation="h", text="Total Sales",
            title="Total Chinook's sales of each country",)
fig.update_traces(marker_color="rgb(207,207,207)")
fig.show()

In [17]:
fig = px.scatter_matrix(data.iloc[:, 1:])
fig.update_layout(height=700, width=970, title='Country Statistics Correlations', title_x=.5)
fig.update_traces(showupperhalf=False, diagonal_visible=False)
fig.show()

- The only highlighting correlation is between **Total Sales** and **No Of Customers** which is strongly positive. But we must be cautious of these result given by small sample size data.
- Regardless it makes sense that Chinook ramping maketing effort on the countries which have large number of population and high lifetime values such as Germany, India, Czech Republic.
- If only one country has been chosen, India makes logical sense as if a high population country (+1.3 billion) and a high lifetime value ($92) 

## Analyse record's sales of each Album

Chinook store is setup a way for allowing customer to purchase in one of two ways for saving their money:
- Purchase a whole album
- Purchase a collection of one or more individual tracks

The collection contains only one or two most popular tracks from each album from record companies, instead of purchasing every track from an album

### Purpose
Write a query that **categorizes each invoice** (purchased by each customer) as either **an album purchase or not**?
<br>
- Means figure out customer purchase whole album or some of tracks of the album

Approach:
- Check each invoice.
- Select all tracks associated with that invoice.
- Find the album associated with any one of those tracks.
- Compare the tracks from the album to those of the invoice.
- If the same tracks and number of tracks are in both, then this is an album purchase.

In [18]:
%%sql 

WITH 
    /* Total number of invoices */
    invoice_count AS (SELECT COUNT(invoice.invoice_id) AS num_invoices
                         FROM invoice),


    album_or_not AS (
        
        /* For each invoice purchase in main query, Compare the existance of all tracks belonged to an invoice (i1)
          to all tracks belonged to an album (a1) --> Figure out customer as i1 purchase whole album or not 
          (NOTE) "a1" and "i1" are in same row of main query*/
     
        SELECT i.invoice_id,
           CASE WHEN (
                       SELECT invoice_line.track_id
                         FROM invoice 
                              INNER JOIN invoice_line 
                                 ON invoice_line.invoice_id = invoice.invoice_id 
                        WHERE invoice.invoice_id = i.invoice_id

                       EXCEPT 

                       SELECT track.track_id
                         FROM album 
                              INNER JOIN track
                                 ON track.album_id = album.album_id 
                        WHERE album.album_id = a.album_id) IS NULL

                      AND 

                      (SELECT track.track_id 
                         FROM album 
                              INNER JOIN track
                                 ON track.album_id = album.album_id
                        WHERE album.album_id = a.album_id

                       EXCEPT 

                       SELECT invoice_line.track_id 
                         FROM invoice 
                              INNER JOIN invoice_line
                                 ON invoice_line.invoice_id = i.invoice_id) IS NULL THEN 'album'
                                                                                    ELSE 'single'
           END AS album_or_not, 
            /* "album_or_not" col represents each invoice purchases a whole album or single */
           SUM(il.unit_price) AS invoice_total
      FROM invoice AS i 
           INNER JOIN invoice_line AS il
              ON il.invoice_id = i.invoice_id

           INNER JOIN track AS t
                 ON t.track_id = il.track_id

           INNER JOIN album AS a 
              ON a.album_id = t.album_id
     GROUP BY i.invoice_id
)
    
SELECT 
    aon.album_or_not AS "Type of purchase",
    COUNT(aon.invoice_id) AS "Number of invoices",
    ROUND(COUNT(aon.invoice_id) / CAST((SELECT COUNT(invoice.invoice_id)
                                          FROM invoice) AS FLOAT), 2) AS "Percentage of invoices",
    ROUND(SUM(aon.invoice_total), 2) AS "Total sales"
FROM album_or_not aon
GROUP BY aon.album_or_not

 * sqlite:///chinook.db
Done.


Type of purchase,Number of invoices,Percentage of invoices,Total sales
album,114,0.19,1461.24
single,500,0.81,3248.19


In [19]:
query = """WITH 
    /* Total number of invoices */
    invoice_count AS (SELECT COUNT(invoice.invoice_id) AS num_invoices
                         FROM invoice),


    album_or_not AS (
        
        /* For each invoice purchase in main query, Compare the existance of all tracks belonged to an invoice (i1)
          to all tracks belonged to an album (a1) --> Figure out customer as i1 purchase whole album or not 
          (NOTE) "a1" and "i1" are in same row of main query*/
     
        SELECT i.invoice_id,
           CASE WHEN (
                       SELECT invoice_line.track_id
                         FROM invoice 
                              INNER JOIN invoice_line 
                                 ON invoice_line.invoice_id = invoice.invoice_id 
                        WHERE invoice.invoice_id = i.invoice_id

                       EXCEPT 

                       SELECT track.track_id
                         FROM album 
                              INNER JOIN track
                                 ON track.album_id = album.album_id 
                        WHERE album.album_id = a.album_id) IS NULL

                      AND 

                      (SELECT track.track_id 
                         FROM album 
                              INNER JOIN track
                                 ON track.album_id = album.album_id
                        WHERE album.album_id = a.album_id

                       EXCEPT 

                       SELECT invoice_line.track_id 
                         FROM invoice 
                              INNER JOIN invoice_line
                                 ON invoice_line.invoice_id = i.invoice_id) IS NULL THEN 'album'
                                                                                    ELSE 'single'
           END AS album_or_not, 
            /* "album_or_not" col represents each invoice purchases a whole album or single */
           SUM(il.unit_price) AS invoice_total
      FROM invoice AS i 
           INNER JOIN invoice_line AS il
              ON il.invoice_id = i.invoice_id

           INNER JOIN track AS t
                 ON t.track_id = il.track_id

           INNER JOIN album AS a 
              ON a.album_id = t.album_id
     GROUP BY i.invoice_id
)
    
SELECT 
    aon.album_or_not AS "Type of purchase",
    COUNT(aon.invoice_id) AS "Number of invoices",
    ROUND(COUNT(aon.invoice_id) / CAST((SELECT COUNT(invoice.invoice_id)
                                          FROM invoice) AS FLOAT), 2) AS "Percentage of invoices",
    ROUND(SUM(aon.invoice_total), 2) AS "Total sales"
FROM album_or_not aon
GROUP BY aon.album_or_not"""

# Connect to database
data = sqlite3.connect("chinook.db")
# Transform query result to dataframe
data = pd.read_sql_query(query, data)

# Use bar chart to visualize
fig = px.bar(data_frame=data, x="Type of purchase",
            y="Number of invoices", title="Purchases of whole album or not")
fig.update_traces(marker_color="rgb(207,207,207)")
fig.show()

As the figure suggest, number of invoice purchased whole album is 114 comprising 19% of total invoices. However, the total amount of money produced by buying whole album is around $1461.24 which accounts for 31\% of total customer sales. It means that Chinook will lose 31\% revenue if they process this stragy proposal.

## Most popular artist

I'd like to determine top 10 most popular artists in Chinook shop

In [20]:
%%sql 
WITH
    track_sold AS (
        SELECT 
            artist_id,
            SUM(track_solds) track_solds
        FROM 
            (SELECT 
                t.album_id,
                COUNT(il.track_id) 'track_solds'
            FROM track t, invoice_line il
            WHERE t.track_id = il.track_id
            GROUP BY t.album_id) AS num_track_sold, album al
        WHERE al.album_id = num_track_sold.album_id
        GROUP BY artist_id )

SELECT 
    ar.name 'Artist',
    ts.track_solds 'Track sold'
FROM track_sold ts, artist ar
WHERE ts.artist_id = ar.artist_id
ORDER BY 2 DESC
LIMIT 10

 * sqlite:///chinook.db
Done.


Artist,Track sold
Queen,192
Jimi Hendrix,187
Nirvana,130
Red Hot Chili Peppers,130
Pearl Jam,129
AC/DC,124
Guns N' Roses,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


In [21]:
query = """WITH
    track_sold AS (
        SELECT 
            artist_id,
            SUM(track_solds) track_solds
        FROM 
            (SELECT 
                t.album_id,
                COUNT(il.track_id) 'track_solds'
            FROM track t, invoice_line il
            WHERE t.track_id = il.track_id
            GROUP BY t.album_id) AS num_track_sold, album al
        WHERE al.album_id = num_track_sold.album_id
        GROUP BY artist_id )

SELECT 
    ar.name 'Artist',
    ts.track_solds 'Track sold'
FROM track_sold ts, artist ar
WHERE ts.artist_id = ar.artist_id
ORDER BY 2 DESC
LIMIT 10"""

# Connect to database
data = sqlite3.connect("chinook.db")
# Convert sql result to dataframe
data = pd.read_sql_query(query, data)

# Sketch horizontal bar graph for number of tracks sold
fig = px.bar(data_frame=data, x="Track sold", y="Artist",
            orientation='h', text="Track sold")
fig.update_traces(marker_color="rgb(207,207,207)")
fig.show()

As a result, "Queen" is the most popular artist in Chinook shop

## Conclusion

Through the analysist we have found the answer to the business questions that will hepl Chinook aid decision-making


With regard to genre popularity, the analysist shows that 53.4% of tracks sold, Rock is the dominant genre platform in USA. Rock is followed by Alternative & Punk and Metal, each with about 12% of tracks sold. This is confirmed by the ten artists with the most tracks sold: Queen, Jimmi Hendrix, Red Hot Chili Peppers, Pearl Jam, AC/DC, Guns N' Roses, Foo Fighters, The Rolling Stones, and Metallica.


The total sales attributed to each Sale Support Agent are as follow: Jane Peacock with \\$1,731.51, Margaret Park with \\$1,584.00, and Steve Johnson with \\$1,393.92. It seems to make sense because Jane Peacock was hired first then Maegaret Park and Steve Johnson


When considering countries in which expand marketing efforts, it makes sense to target countries with high population and high lifetime values. One country that satisfies both requirements is India. India has a population of over 1.3 billion and customer average lifetime values is total \\$91.58. Other countries, such as the US, Germany, Brazil, Portugal, and Czech Republic meet one, not both of the criteria.


I have figured out album sales makes up 19% of total sales market. For this reason, it is not recommended to eliminate whole album purchase option in favor of focusing only in single purchase option. Doing so can potentially lead to a loss of \\$1,461.24.